# Assignment 2

In [2]:

# Import necessary packages
import tarfile
import os
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import locale

In [12]:
file_path = "kungalv_slutpriser.tar.gz"

if file_path.endswith("tar.gz"):
    tar = tarfile.open(file_path, "r:gz")
    tar.extractall()
    tar.close()
elif file_path.endswith("tar"):
    tar = tarfile.open(file_path, "r:")
    tar.extractall()
    tar.close()



In [3]:
# Helper Functions
def clean_date(date):
    locale.setlocale(locale.LC_ALL, 'sv_SE')
    datetime_object = datetime.strptime(date, '%d %B %Y')
    return datetime_object.strftime("%Y-%m-%d")



In [67]:
import re 



with open("kungalv_slutpriser/kungalv_slutpris_page_01.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

# date of sale
listings = soup.findAll('li', class_ = 'sold-results__normal-hit')

listing_elements = []
for listing in listings:
    listing_soup = BeautifulSoup(str(listing), 'html.parser')

    area = listing_soup.find('div', {'class' : 'sold-property-listing__subheading sold-property-listing__area'}).text.strip()
    # TO DO PRITISH: Works but needs regex implementation
    area = re.sub(r'\s', ' ', area.strip())
    arealist = re.findall(r'\d+', area)

    if len(arealist) == 3:
        boarea = int(arealist[0])
        biarea = int(arealist[1])
        rooms = int(arealist[2])
        livingarea = boarea+biarea
    elif len(arealist) == 2:
        livingarea = int(arealist[0])
        rooms = int(arealist[1])


    date = listing_soup.find('span', {'class' : 'hcl-label hcl-label--state hcl-label--sold-at'}).text.strip()
   
    match = re.search(r'\d', date)  # Search for the first number
   
    date_of_sale = clean_date((date[match.start():]))
    
    address = listing_soup.find('h2', {'class' : 'sold-property-listing__heading qa-selling-price-title hcl-card__title'}).text.strip()
    
    location = listing_soup.find('div', {'class' : 'sold-property-listing__location'}).text
    location = re.sub(r'\s+', ' ', location.split("VillaVilla")[1].strip())
    
    listingarea = listing_soup.find('div', {'class' : 'sold-property-listing__subheading sold-property-listing__area'}).text.strip()
    
    plotarea = listing_soup.find('div', {'class' : 'sold-property-listing__land-area'}).text.strip()
    plotarea = int("".join(map(str,re.findall(r'\b\d+\b', plotarea))))

    closingprice = listing_soup.find('span',{'class' : 'hcl-text hcl-text--medium'}).text.strip()
    closingprice = int("".join(map(str,re.findall(r'\b\d+\b', closingprice))))
   # print(area, dateofsale, address, location, listingarea, plotarea, closingprice)
    
    # Append the attributes for each listing to a list
    listing_info = [date_of_sale, address, location, livingarea, rooms, plotarea, closingprice]
    
    # Append the list of attributes to the overall list of listings
    listing_elements.append(listing_info)
    
#print(*listing_elements,sep = "\n")

df = pd.DataFrame(listing_elements, columns=(('date of sale', 'address','location','livingarea','rooms','plotarea','closingprice')))
df.info()
 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date of sale  50 non-null     object
 1   address       50 non-null     object
 2   location      50 non-null     object
 3   livingarea    50 non-null     int64 
 4   rooms         50 non-null     int64 
 5   plotarea      50 non-null     int64 
 6   closingprice  50 non-null     int64 
dtypes: int64(4), object(3)
memory usage: 2.9+ KB


In [ ]:
   

    
#address
    
#location
    
#area in the form (boarea + biarea)

#number of rooms
    
#area of the plot
    

#closing price
